# Лаб 1. Корреляционная система

При создании задания использовались Visual Studio Code и Python 3.8.

## Теория

Данные для рекомендательных систем представляют из себя матрицу $R$ юзер-айтемов, в строках которой находятся пользователи, а в столбцах объекты. В ячейках матрицы находятся оценки пользователей (или какая-то другая метрика $-$ просмотры, покупки и т.д.):

$$
R = \begin{pmatrix}
   r_{1 1} & r_{1 2} & ...     &    \\
   r_{2 1} & r_{2 1} & ...     &     \\
   ...     & ...     & r_{u i} & ... \\
           &         & ...     &     \\
\end{pmatrix}
$$

Матрица $R$ сильно разрежена, то есть большая часть ячеек пуста, поскольку каждый пользователь взаимодействовал с очень малым числом объектов. Задача $-$ предсказать значения в пустых ячейках, то есть получить новую заполненную матрицу $\hat R$, максимально похожую на $R$.

### Обозначения

- $U - $ множество пользователей;
- $I - $ множество айтемов (объектов);
- $I(u)$, где $u \in U -$ множество айтемов, которое оценил пользователь $u$. То есть такие айтемы, для которых в строке $u$ матрицы $R$ не пусто. Аналогично $I(u, v) -$ множество объектов, которые оценили и $u$ и $v$;
- $U(u)$, где $u \in U -$ Множество пользователей, оценивали то же что и $u$, то есть $I(u) \cap I(v) \ne \empty$, где $v \in U(u)$;
- $r_u$, где $u \in U -$ строка матрицы $R$, соответствующая пользователю $u$;
- $r_i$, где $i \in I -$ столбец матрицы $R$, соответствующий айтему $i$;
- $\bar r_u$, где $u \in U -$ среднее значение по всем заполненным оценкам пользователя $u$.


### Корреляционная модель

Один из самых простых способов $-$ использовать взвешенное среднее. То есть, чтобы определить оценку пользователя $u$ айтема $i$, надо усреднить оценки всех пользователей, посмотревших этот фильм (user-based подход). При этом будем учитывать оценку пользователя с большим коэффициентом если пользователь "похож" на нашего:

$$
\hat{r}_{ui} = \frac{
        \sum_{v \in U(u)} S(u, v)\cdot r_v
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Здесь $S$ это функция близости, которая тем больше, чем более "похожи" пользователи друг на друга.

Поскольку оценки разных пользователей могут отличаться $-$ кто-то ставит всем фильмам 9 и 10, а кто-то 0 и 1, можно попытаться устранить проблему, предсказывая не само значение $r_u$, а отклонение от среднего значения $(r_u - \bar{r}_u)$:

$$
\hat{r}_{ui} = \bar{r}_{u} + \frac{
        \sum_{v \in U(u)} S(u, v) \cdot (r_v - \bar r_v)
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Аналогично определяется item-based модель, только усреднение оценок происходит не по пользователям, оценившим этот объект, а по объектам, которые оценил данный пользователь. Теперь $S$ будет функцией близости двух айтемов:

$$
\hat{r}_{ui} = \bar{r}_{i} + \frac{
        \sum_{j \in I(i)} S(i, j) \cdot (r_i - \bar r_i)
    } {
        \sum_{j \in I(i)} S(i, j)
    }
$$


### Функция сходства

Будем рассматривать функцию сходства двух юзером для user-based модели, функции сходства для айтемов определяется аналогично.

#### Косинусная мера сходства

Считаем косинус угла между пользователями в пространстве определяющих их векторов. То есть берём скалярное произведение и делим на длины векторов. Единственное, на что следует обратит внимание, что берутся не все оценки пользователей, а только для тех айтемов, которые они оба оценили:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} r_{ui}r_{vi}
    }{
        \sqrt{\sum_{i \in I(u)} r_{ui}^2}
        \sqrt{\sum_{i \in I(v)} r_{vi}^2}
    }
$$

#### Корреляция Пирсона

"Коэффициент корреляции Пирсона - это ковариация двух переменных, делённая на произведение их стандартных отклонений."

По смыслу примерно то же что предыдущая, но теперь мы опять учитываем не сами значения, а отклонения от среднего:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} (r_{ui} - \bar r_u) (r_{vi} - \bar r_v)
    }{
        \sqrt{\sum_{i \in I(u, v)} (r_{ui} - \bar r_u)^2}
        \sqrt{\sum_{i \in I(u, v)} (r_{vi} - \bar r_v)^2}
    }
$$

### Функция сходства для бинарных данных

А некоторых задачах взаимодействие пользователя с предметом либо есть, либо нет, например, факт покупки чего-то в интернет магазине. Для таких задач существуют отдельные методы.

![Диаграмма пересечения классов](./images/IU.png)

Пусть $|I(u)| = p$, а $|I(v)| = q$, то есть количество объектов, оцененных $u$ равно $p$, а количество объектов, оцененных $v$ равно $q$. А размер пересечения $|I(u) \cap I(v)| = |I(u, v)| = m$

#### Мера близости Жаккара

Очевидно:

$$
S(u, v) = \frac{
        |I(u) \cap I(v)|
    }{
        |I(u) \cup I(v)|
    }
$$

#### Точный тест Фишера

Если предположить что у пользователей $u$ и $v$ оценки генерируются независимо друг от друга, то можно посчитать вероятность того, что размер пересечения будет именно таким, каким он получился $P \{ |I(u) \cap I(v)| = m \}$. Чем меньше эта вероятность, тем больше схожесть пользователей между собой:

$$
S(u, v)
    = -\log P \{ |I(u) \cap I(v)| = m \}
    = -\log \frac{C_{q}^m C_{|I| - q}^{p-m}}{C_{|I|}^{p}}
$$


## Задание

Дан датасет с оценками пользователями фильмов. Реализуйте алгоритм в соответствии с вашим вариантом и порекомендуйте себе фильмы. Сделайте выводы.

В датасете фильмы оценены по пятибальной шкале. Если в вашем варианте используется функция сходства для бинарных данных, используйте факт просмотра фильма (наличие оценки вообще).

### Варианты

1. User-based подход, Косинусная мера сходства.
2. User-based подход, Корреляция Пирсона.
3. User-based подход, Мера близости Жаккара.
4. Item-based подход, Косинусная мера сходства.
5. Item-based подход, Корреляция Пирсона
6. Item-based подход, Мера близости Жаккара.


## Код

Устанавливаем библиотеки

In [1]:
%pip install pandas tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Импорты

In [2]:
# Пандас нам нужен для загрузки csv и работы с матрицей
import pandas as pd
# Это библиотека для визуализации прогресса в питоновском ноутбуке
from tqdm.notebook import tqdm

import math
import time
import numpy as np

Загружаем датасет

In [84]:
# Сам датасет
ratings = pd.read_csv('./ml-latest-small/ratings.csv', delimiter=',')
# Оставляем в таблице только нужные столбцы
ratings = ratings.loc[:, ["userId", "movieId", "rating"]]

# Строка в таблице в датасета это id пользователя, id фильма, рейтинг
# На id пользователей нам плевать, а фильмы хочется смотреть по названиям,
# поэтому загружаем табличку сопоставления названий фильмов их id
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')

In [85]:
# Делаем таблицу для преобразования имени в id
title_to_id = movies.loc[:, ["title", "movieId"]]
title_to_id.set_index("title", inplace=True)

# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

# Проверяем что всё работает
print(title_to_id.loc[["Iron Man (2008)", "Doctor Who: A Christmas Carol (2010)"], :])
print(id_to_title.loc[[59315, 147376], :])


                                      movieId
title                                        
Iron Man (2008)                         59315
Doctor Who: A Christmas Carol (2010)   147376
                                        title
movieId                                      
59315                         Iron Man (2008)
147376   Doctor Who: A Christmas Carol (2010)


In [97]:
# Мы хотим получить для себя какую-то рекомендацию,
# для этого оценим несколько фильмов по пятибальной шкале
# (аккуратно копипастим имена из датасета)
my_ratings = {
    "Doctor Who: A Christmas Carol (2010)": 6.0,
    "Star Wars: Episode IV - A New Hope (1977)": 5.0,
    "Star Wars: Episode I - The Phantom Menace (1999)": 5.0,
    "LEGO Batman: The Movie - DC Heroes Unite (2013)": 4.0,
    "Lord of the Rings: The Fellowship of the Ring, The (2001)": 3.0,
    "Back to the Future (1985)": 5.0,
    "Iron Man (2008)": 5.0,
    "Avengers, The (2012)": 3.0,
    "Star Wars: The Last Jedi (2017)": 1.0
}

# Даём нашему юзеру id, которого нет в датасете
my_user_id = 666

# Докидываем свои оценки в датасет
for m, r in my_ratings.items():
    mid = title_to_id.loc[m]["movieId"]
    row = pd.DataFrame([[my_user_id, mid, r]], columns=["userId", "movieId", "rating"])
    ratings = pd.concat([ratings, row])

In [117]:
def getUserAvgRate(userId) :
    userRankedMoviesArray = ratings.loc[ratings['userId'] == userId, ['rating']].to_numpy()
    return np.average(userRankedMoviesArray, axis=0)
    
def getMatrixElem(userId, movieId) :
    elem = ratings.loc[(ratings['userId'] == userId) & (ratings['movieId'] == movieId), ['rating']].values
    return elem[0][0] if elem.size else 0

def getSelfAndGeneralMoviesForUsers(firstUserId, secondUserId) :
    firstUserMovies = ratings.loc[ratings['userId'] == firstUserId, ['movieId']]
    secondUserMovies = ratings.loc[ratings['userId'] == secondUserId, ['movieId']]
    generalMovies = pd.merge(firstUserMovies, secondUserMovies)
    return firstUserMovies.values.flatten(), secondUserMovies.values.flatten(), generalMovies.values.flatten()

def getStandardDeviation(userId, userAvgRate, userMoviesArray) :
    tempVec = []
    for movieId in userMoviesArray:
        tempVec.append(getMatrixElem(userId, movieId) - userAvgRate)
    return np.linalg.norm(tempVec)


def getPirsonСorrelationCoeff(firstUserId, secondUserId) :
    fAvgRate = getUserAvgRate(firstUserId)
    sAvgRate = getUserAvgRate(secondUserId)

    fMovieArray, sMovieArray, generalMovieArray = getSelfAndGeneralMoviesForUsers(firstUserId, secondUserId)

    covariation = 0
    for movieId in generalMovieArray:
        covariation = covariation + (getMatrixElem(firstUserId, movieId) - fAvgRate) * (getMatrixElem(secondUserId, movieId) - sAvgRate)

    div = getStandardDeviation(firstUserId, fAvgRate, fMovieArray) * getStandardDeviation(secondUserId, sAvgRate, sMovieArray)
    result = covariation / div
    return result



In [120]:
ratings['movieId'].unique().size

9724

In [115]:
# Тут реализуем функцию предсказания.
# Или класс. Главное - предсказать.

def predict(user_id, movie_id):
    fMovies = ratings.loc[ratings['userId'] == user_id, ['movieId']].values
    fMovies = fMovies.flatten()
    sameUsers = ratings.loc[ratings['movieId'].isin(fMovies) == True]['userId'].unique()

    userAvg = getUserAvgRate(user_id)
    num = 0
    div = 0
    for anotherUserId in sameUsers:
        pirsonCoeff = getPirsonСorrelationCoeff(user_id, anotherUserId)[0]
        num = num + pirsonCoeff * (getMatrixElem(anotherUserId, movie_id) - getUserAvgRate(anotherUserId))
        div = div + pirsonCoeff

    return userAvg + num / div


Даём предсказание для каждого фильма

In [130]:
result = {}
all = ratings['movieId'].unique().size
i = 0
for m in ratings["movieId"].unique():
    result[m] = predict(my_user_id, m)
    i = i + 1
    print(i / all * 100, "prc done")
    print(result[m], '  ==>  ', movies.loc[movies['movieId'] == m, ['title']].values[0][0])
result = {k: v for k, v in result.items()}

0.010283833813245578 prc done
[2.3138937]   ==>   Toy Story (1995)
0.020567667626491155 prc done
[0.96725051]   ==>   Grumpier Old Men (1995)
0.030851501439736733 prc done
[1.58046518]   ==>   Heat (1995)
0.04113533525298231 prc done
[0.84007167]   ==>   Seven (a.k.a. Se7en) (1995)
0.051419169066227885 prc done
[1.16722387]   ==>   Usual Suspects, The (1995)
0.061703002879473466 prc done
[0.51170589]   ==>   From Dusk Till Dawn (1996)
0.07198683669271905 prc done
[0.31794038]   ==>   Bottle Rocket (1996)
0.08227067050596462 prc done
[1.04736963]   ==>   Braveheart (1995)
0.0925545043192102 prc done
[0.81418117]   ==>   Rob Roy (1995)
0.10283833813245577 prc done
[0.53055552]   ==>   Canadian Bacon (1995)
0.11312217194570137 prc done
[0.57090443]   ==>   Desperado (1995)
0.12340600575894693 prc done
[0.78838494]   ==>   Billy Madison (1995)
0.1336898395721925 prc done
[0.14539661]   ==>   Clerks (1994)
0.1439736733854381 prc done
[0.35806294]   ==>   Dumb & Dumber (Dumb and Dumber) (199

KeyboardInterrupt: 

Выводим результат

In [ ]:
# Преобразуем id фильмов в нормальные названия
human_readable_result = {}
for m, v in result.items():
    title = id_to_title.loc[m]["title"]
    human_readable_result[title] = v

# Сортируем массив с результатами по убыванию
sorted_result = {k: v for k, v in sorted(human_readable_result.items(), key=lambda item: item[1], reverse=True) if v != 0.0}

# Выводим первые 20 рекомендаций
for m, v in list(sorted_result.items())[:20]:
    print(m, ": ", v)


## Выводы

Объясните полученный результат: